In [495]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [496]:
import numpy as np
import torch
import pprint
import matplotlib.pyplot as plt
import sys
import pickle
import argparse
import logging
import os

# from ginkgo import invMass_ginkgo
# from invMass_ginkgo import *
from invMass_ginkgo_node import *
from ginkgo.utils import get_logger


logger = get_logger(level = logging.WARNING)
fh = logging.FileHandler('spam.log')
fh.setLevel(logging.WARNING)
logger.addHandler(fh)


"""
create node class
understand original output
simplify recursion logic further

define typology of a tree
access data on each node

call algo to generate tree
have a fully defined tree with 4D vectors for each node

generate likelihood for each tree 
compute eq (12)
eq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r

"""

'\ncreate node class\nunderstand original output\nsimplify recursion logic further\n\ndefine typology of a tree\naccess data on each node\n\ncall algo to generate tree\nhave a fully defined tree with 4D vectors for each node\n\ngenerate likelihood for each tree \ncompute eq (12)\neq(14) simplifies to eq(8), apply 8 repeatedly when matching l and r\n\n'

In [497]:
"""Parameters"""


rate2 = torch.tensor(8.)

# Parameters to get ~<10 constituents to test the trellis algorithm
pt_min = torch.tensor(4.**2)

### Physics inspired parameters to get ~ between 20 and 50 constituents
W_rate = 3.
QCD_rate = 1.5

QCD_mass = 30.
class ginkgo_simulator():
    def __init__(self,
                 rate,
                 pt_cut,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate= 1.5,
                ):
        
        self.root_rate = root_rate
        self.rate = rate
        self.pt_cut = pt_cut
        self.M2start = torch.tensor(M2start) # mass squared to start with
        self.Nsamples = Nsamples
        self.minLeaves = minLeaves
        self.maxLeaves = maxLeaves
        self.maxNTry = maxNTry
        self.jetType = jetType # W or QCD 
        self.jetM = np.sqrt(M2start) # mass to start with
        self.jetdir = np.array([1,1,1])
        self.jetP = jetP
        self.jetvec = self.jetP * self.jetdir / np.linalg.norm(self.jetdir)
        self.jet4vec = np.concatenate(([np.sqrt(self.jetP ** 2 + self.jetM ** 2)], self.jetvec))
        logger.debug(f"jet4vec = {self.jet4vec}")
        
        if jetType == "W":
            # defined in paper, W jets have a different root rate
            self.rate=torch.tensor([self.root_rate,self.rate])
        elif jetType == "QCD":
            # QCD jets maintain the same rate throughout
            self.rate=torch.tensor([self.rate,self.rate])
        else:
            raise ValueError("Choose a valid jet type between W or QCD")



    def simulator(self):

        simulator = Simulator(jet_p = self.jet4vec,
                                         pt_cut = float(self.pt_cut),
                                         Delta_0 = self.M2start,
                                         M_hard = self.jetM ,
                                         num_samples = int(self.Nsamples),
                                         minLeaves = int(self.minLeaves),
                                         maxLeaves = int(self.maxLeaves),
                                         maxNTry = int(self.maxNTry)
                                         )
        return simulator
       
    def generate(self):
        
        simulator = self.simulator()
        jet_list = simulator(self.rate)

        logger.debug(f"---"*10)
        logger.debug(f"jet_list = {jet_list}")
        
        return jet_list

In [498]:
# the range of leaves that you would consider as valid generations
minLeaves = 3
maxLeaves = 100

# number of jets you wish to generate
Nsamples = 1

# exponential rate parameter
rate = 1.5

# mass squared cut off to yield leaves
pt_cut =  torch.tensor(1.1**2)

# mass squared to start with
# M2start = 80.**2
M2start = 5.**2

# the maximum times you are willing to try to get Nsamples
maxNTry = 1

# jetP=400.
jetP=4.

In [499]:
jetType ="QCD"


ginkgo = ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP)

QCD_jets = ginkgo.generate()

Node 0
 Vec4: [6.40312424 2.30940108 2.30940108 2.30940108]
 Decay Rate: 1
 Mass Squared: tensor(25.)
 Log Likelihood: -7.650441827693098
 DIJ List: -7.650441827693098 0.19047555067053398 1.4539823568807873 0.6635261316163508

Node 1
 Vec4: [2.98711693 0.08539945 0.67011776 1.92381047]
 Decay Rate: tensor(0.5196)
 Mass Squared: tensor(4.7655)
 Log Likelihood: -4.587870627203055
 DIJ List: -4.587870627203055 3.1480092564541358 0.7879383234230382 0.03225306383200582

Node 2
 Vec4: [3.41600731 2.22400163 1.63928331 0.38559061]
 Decay Rate: tensor(0.1555)
 Mass Squared: tensor(3.8870)
 Log Likelihood: -3.8790810401821814
 DIJ List: -3.8790810401821814 0.1782056094779521 0.8711137718539863 0.2751745565708454

Node 3
 Vec4: [ 0.36714181 -0.06288246  0.19229999  0.10338323]
 Decay Rate: tensor(0.6195)
 Mass Squared: tensor(0.0832)
 Log Likelihood: 0
 DIJ List:

Node 4
 Vec4: [2.61997503 0.1482819  0.47781776 1.82042719]
 Decay Rate: tensor(0.6925)
 Mass Squared: tensor(3.3000)
 Log Likelihood

In [500]:
root_rate = 4.
jetType ="W"


ginkgo= ginkgo_simulator(
                 rate,
                 pt_cut ,
                 M2start,
                 Nsamples,
                 minLeaves,
                 maxLeaves,
                 maxNTry,
                 jetType, 
                 jetP,
                 root_rate)

W_jets = ginkgo.generate()

Node 0
 Vec4: [6.40312424 2.30940108 2.30940108 2.30940108]
 Decay Rate: 1
 Mass Squared: tensor(25.)
 Log Likelihood: -7.550832498064582
 DIJ List: -7.550832498064582 0.4320988781871568 5.910654056249487 5.017925319555888

Node 1
 Vec4: [5.00645209 1.93560695 3.15156539 3.16673684]
 Decay Rate: tensor(0.0543)
 Mass Squared: tensor(1.3574)
 Log Likelihood: -2.6411049659634314
 DIJ List: -2.6411049659634314 0.016097708167630422 0.10904843511736505 0.14934691835840982

Node 2
 Vec4: [ 1.39667215  0.37379413 -0.84216431 -0.85733576]
 Decay Rate: tensor(0.0249)
 Mass Squared: tensor(0.3667)
 Log Likelihood: 0
 DIJ List:

Node 3
 Vec4: [1.51475025 0.87846865 0.7732008  0.95919233]
 Decay Rate: tensor(0.0036)
 Mass Squared: tensor(0.0049)
 Log Likelihood: 0
 DIJ List:

Node 4
 Vec4: [3.49170172 1.05713825 2.37836451 2.20754443]
 Decay Rate: tensor(0.4539)
 Mass Squared: tensor(0.5446)
 Log Likelihood: 0
 DIJ List:

  ┌2
 0┤
  │ ┌3
  └1┤
    └4


In [501]:
def llh(pL, pR, t_cut, lam):
    """
    Take two nodes and return the splitting log likelihood
    """
    tL = pL[0] ** 2 - np.linalg.norm(pL[1::]) ** 2
    tR = pR[0] ** 2 - np.linalg.norm(pR[1::]) ** 2


    pP = pR + pL ## eq (5)


    """Parent invariant mass squared"""
    tp = pP[0] ** 2 - np.linalg.norm(pP[1::]) ** 2

    if tp<=0 or tL<0 or tR<0:
        return - np.inf

    """ We add a normalization factor -np.log(1 - np.exp(- lam)) because we need the mass squared to be strictly decreasing. This way the likelihood integrates to 1 for 0<t<t_p. All leaves should have t=0, this is a convention we are taking (instead of keeping their value for t given that it is below the threshold t_cut)"""
    def get_logp(tP_local, t, t_cut, lam):


        if t > t_cut:
            """ Probability of the shower to stop F_s"""
            return -np.log(1 - np.exp(- (1. - 1e-3)*lam)) + np.log(lam) - np.log(tP_local) - lam * t / tP_local

        else: # For leaves we have t<t_cut
            t_upper = min(tP_local,t_cut) #There are cases where tp2 < t_cut
            log_F_s = -np.log(1 - np.exp(- (1. - 1e-3)*lam)) + np.log(1 - np.exp(-lam * t_upper / tP_local))
            return log_F_s


    if tp <= t_cut:
        "If the pairing is not allowed"
        logLH = - np.inf

    elif tL >=(1 - 1e-3)* tp or tR >=(1 - 1e-3)* tp:
        # print("The pairing is not allowed because tL or tR are greater than tP")
        logLH = - np.inf

    elif np.sqrt(tL) + np.sqrt(tR) > np.sqrt(tp):
        print("Breaking invariant mass inequality condition")
        logLH = - np.inf


    else:
        """We sample a unit vector uniformly over the 2-sphere, so the angular likelihood is 1/(4*pi)"""

        tpLR = (np.sqrt(tp) - np.sqrt(tL)) ** 2
        tpRL = (np.sqrt(tp) - np.sqrt(tR)) ** 2

        logpLR = np.log(1/2)+ get_logp(tp, tL, t_cut, lam) + get_logp(tpLR, tR, t_cut, lam) #First sample tL
        logpRL = np.log(1/2)+ get_logp(tp, tR, t_cut, lam) + get_logp(tpRL, tL, t_cut, lam) #First sample tR

        logp_split = logsumexp(np.asarray([logpLR, logpRL]))

        logLH = (logp_split + np.log(1 / (4 * np.pi)) ) ## eq (8)

    return logLH, tp

In [502]:
from node import *
import numpy as np

# finish function
# test function against actual values
# modify code to output 

"""


why is it left leaning???

3 cases

2 leaves
1 leave 1 internal
2 internal


tenserflow version (1.15 or something)



"""
cut_off = pt_cut # cut off rate defined previously
leaves = [(index, i) for index, i in enumerate(QCD_jets) if i.left is None and i.right is None]
print(*sorted([i[0] for i in leaves]))

print()
def testRecontruct(parent, left, right, cut_off):
    parentDecayRate = parent.decay_rate

    def reconstruct(left, right, cut_off):
        nonlocal parentDecayRate
        parVec4 = left.vec4 + right.vec4
        parLH, parDelta = llh(left.vec4, right.vec4, cut_off, parentDecayRate)
        parent = jetNode(vec4 = parVec4,
                         left = left,
                         right = right,
                         decay_rate = parentDecayRate, # template value
                         delta = parDelta,
                         logLH = parLH
                         )
        return parent
    res = reconstruct(left, right, cut_off)
    
    print(parent.vec4, res.vec4)
    print(parent.decay_rate, res.decay_rate)
    print(parent.delta, res.delta)
    print(parent.logLH, res.logLH)
    print()
    return res

# testRecontruct(QCD_jets[0], QCD_jets[1], QCD_jets[2], cut_off)

def rec_test(node, cut_off):
    if node.right and node.left:
        testRecontruct(node, node.left, node.right, cut_off)
        rec_test(node.left, cut_off)
        rec_test(node.right, cut_off)
rec_test(QCD_jets[0], cut_off)


    

3 5 6 7 8

[6.40312424 2.30940108 2.30940108 2.30940108] [6.40312424 2.30940108 2.30940108 2.30940108]
1 1
tensor(25.) 24.999999999999993
-7.650441827693098 -7.650441827693098

[2.98711693 0.08539945 0.67011776 1.92381047] [2.98711684 0.08539945 0.67011774 1.92381042]
tensor(0.5196) tensor(0.5196)
tensor(4.7655) 4.765469657908733
-4.587870627203055 -4.587870627203055

[2.61997503 0.1482819  0.47781776 1.82042719] [2.61997503 0.1482819  0.47781776 1.82042719]
tensor(0.6925) tensor(0.6925)
tensor(3.3000) 3.300016669281527
-3.49041876152999 -3.49041876152999

[3.41600731 2.22400163 1.63928331 0.38559061] [3.41600741 2.22400169 1.63928336 0.38559062]
tensor(0.1555) tensor(0.1555)
tensor(3.8870) 3.8869930566110478
-3.8790810401821814 -3.8790810401821814

